<a href="https://colab.research.google.com/github/shelly-hub/deep-learning-challenge/blob/main/Alphabet_soup_optimised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
# application_df = application_df.drop(columns = ['EIN', 'NAME'])
application_df = application_df.drop(columns = ['NAME'])
application_df.head()

,EIN,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

EIN                       34299
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
# Need to change value counts to list
app_counts = application_df['APPLICATION_TYPE'].value_counts()
application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts()[app_counts < 500].index.tolist()
# print(application_types_to_replace)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_counts = application_df['CLASSIFICATION'].value_counts()
class_filterone = application_df['CLASSIFICATION'].value_counts()[class_counts > 1]
class_filterone

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = application_df['CLASSIFICATION'].value_counts()[class_counts < 1000].index.tolist()
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
application_df['ASK_AMT'].astype('int')
application_df.dtypes

EIN                        int64
APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                    int64
IS_SUCCESSFUL              int64
dtype: object

In [10]:
# Choose a cutoff value and create a list of ASK_AMT to be replaced

min5000 = application_df['ASK_AMT'][application_df['ASK_AMT'] == 5000].index.tolist()

lessthan10000 = application_df['ASK_AMT'][(application_df['ASK_AMT'] < 10000) & (application_df['ASK_AMT'] > 5000)].index.tolist()
lessthan100000 = application_df['ASK_AMT'][(application_df['ASK_AMT'] < 100000) & (application_df['ASK_AMT'] > 10000)].index.tolist()
lessthanmillions = application_df['ASK_AMT'][(application_df['ASK_AMT'] < 1000000) & (application_df['ASK_AMT'] > 100000)].index.tolist()
millions = application_df['ASK_AMT'][application_df['ASK_AMT'] > 1000000].index.tolist()

# Replace in dataframe
for five in min5000:
    application_df['ASK_AMT'] = application_df['ASK_AMT'].replace(five,"min5000")
for ten in lessthan10000 :
    application_df['ASK_AMT'] = application_df['ASK_AMT'].replace(ten,"Lessthan10000")
for hundred in lessthan100000 :
    application_df['ASK_AMT'] = application_df['ASK_AMT'].replace(hundred,"Lessthan100000")
for thousand in lessthanmillions :
    application_df['ASK_AMT'] = application_df['ASK_AMT'].replace(thousand,"LessthanMillions")
for mil in millions :
    application_df['ASK_AMT'] = application_df['ASK_AMT'].replace(mil,"MorethanMillions")

# Check to make sure binning was successful
application_df['ASK_AMT']


0               min5000
1                108590
2               min5000
3        Lessthan100000
4                142590
              ...      
34294           min5000
34295           min5000
34296           min5000
34297           min5000
34298          36500179
Name: ASK_AMT, Length: 34299, dtype: object

In [11]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)
application_df.head()

,EIN,STATUS,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,ASK_AMT_3391919220,ASK_AMT_4653011914,ASK_AMT_5591584994,ASK_AMT_8556638692,ASK_AMT_8597806340,ASK_AMT_Lessthan10000,ASK_AMT_Lessthan100000,ASK_AMT_LessthanMillions,ASK_AMT_MorethanMillions,ASK_AMT_min5000
0,10520599,1,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,10531628,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,10547893,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
3,10553066,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,10556103,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
# Split our preprocessed data into our features and target arrays
# Make sure to change list dataframe to values
y = application_df['IS_SUCCESSFUL'].values
X = application_df.drop(columns = 'IS_SUCCESSFUL').values


# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.25, stratify = y)

In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

## **MODEL 1** - ADDING "EIN" COLUMN ONLY

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =50
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units = hidden_nodes_layer1, input_dim = number_input_features, activation = 'relu')
)
# Second hidden layer
nn.add(
    tf.keras.layers.Dense(units = hidden_nodes_layer2, activation = 'relu')
)

# Output layer
nn.add(
    tf.keras.layers.Dense(units = 1, activation = 'sigmoid')
)

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 50)                2250      
                                                                 
 dense_1 (Dense)             (None, 30)                1530      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 3,811
Trainable params: 3,811
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compile the model
nn.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs = 100)

Epoch 1/100
804/804 [==============================] - 6s 4ms/step - loss: 0.5711 - accuracy: 0.7199
Epoch 2/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5525 - accuracy: 0.7322
Epoch 3/100
804/804 [==============================] - 5s 6ms/step - loss: 0.5493 - accuracy: 0.7328
Epoch 4/100
804/804 [==============================] - 4s 5ms/step - loss: 0.5467 - accuracy: 0.7352
Epoch 5/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5447 - accuracy: 0.7364
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5441 - accuracy: 0.7368
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5431 - accuracy: 0.7377
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5416 - accuracy: 0.7365
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5409 - accuracy: 0.7385
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5408 - accura

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test, verbose = 2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5492 - accuracy: 0.7310 - 600ms/epoch - 2ms/step
Loss: 0.5492396950721741, Accuracy: 0.7309620976448059


## **MODEL 2** ADDING "EIN" COLUMN & BINNING "ASK_AMT" COLUMN

In [18]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =80
hidden_nodes_layer2 =30

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(
    tf.keras.layers.Dense(units = hidden_nodes_layer1, input_dim = number_input_features, activation = 'relu')
)
# Second hidden layer
nn2.add(
    tf.keras.layers.Dense(units = hidden_nodes_layer2, activation = 'relu')
)

# Second hidden layer
nn2.add(
    tf.keras.layers.Dense(units = 10, activation = 'relu')
)

# Output layer
nn2.add(
    tf.keras.layers.Dense(units = 1, activation = 'sigmoid')
)

# Check the structure of the model
nn2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 80)                531920    
                                                                 
 dense_4 (Dense)             (None, 30)                2430      
                                                                 
 dense_5 (Dense)             (None, 10)                310       
                                                                 
 dense_6 (Dense)             (None, 1)                 11        
                                                                 
Total params: 534,671
Trainable params: 534,671
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Compile the model
nn2.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

# Train the model
fit_model = nn2.fit(X_train_scaled, y_train, epochs =100)

Epoch 1/100
804/804 [==============================] - 9s 10ms/step - loss: 0.5883 - accuracy: 0.6955
Epoch 2/100
804/804 [==============================] - 7s 9ms/step - loss: 0.5057 - accuracy: 0.7620
Epoch 3/100
804/804 [==============================] - 8s 10ms/step - loss: 0.4386 - accuracy: 0.7933
Epoch 4/100
804/804 [==============================] - 7s 9ms/step - loss: 0.4319 - accuracy: 0.7967
Epoch 5/100
804/804 [==============================] - 8s 10ms/step - loss: 0.4307 - accuracy: 0.7979
Epoch 6/100
804/804 [==============================] - 8s 10ms/step - loss: 0.4281 - accuracy: 0.7982
Epoch 7/100
804/804 [==============================] - 7s 9ms/step - loss: 0.4272 - accuracy: 0.7990
Epoch 8/100
804/804 [==============================] - 8s 10ms/step - loss: 0.4265 - accuracy: 0.7995
Epoch 9/100
804/804 [==============================] - 10s 12ms/step - loss: 0.4257 - accuracy: 0.8004
Epoch 10/100
804/804 [==============================] - 8s 9ms/step - loss: 0.4250 -

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.6572 - accuracy: 0.7243 - 736ms/epoch - 3ms/step
Loss: 0.6571983098983765, Accuracy: 0.7243148684501648


## **MODEL 3** - ADDING "EIN" COLUMN only


In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =200
hidden_nodes_layer2 =100

nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(
    tf.keras.layers.Dense(units = hidden_nodes_layer1, input_dim = number_input_features, activation = 'relu')
)
# Second hidden layer
nn3.add(
    tf.keras.layers.Dense(units = hidden_nodes_layer2, activation = 'tanh')
)

# Output layer
nn3.add(
    tf.keras.layers.Dense(units = 1, activation = 'sigmoid')
)

# Check the structure of the model
nn3.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 200)               9000      
                                                                 
 dense_15 (Dense)            (None, 100)               20100     
                                                                 
 dense_16 (Dense)            (None, 1)                 101       
                                                                 
Total params: 29,201
Trainable params: 29,201
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Compile the model
nn3.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

# Train the model
fit_model = nn3.fit(X_train_scaled, y_train, epochs = 50)

Epoch 1/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5715 - accuracy: 0.7212
Epoch 2/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5565 - accuracy: 0.7290
Epoch 3/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5519 - accuracy: 0.7318
Epoch 4/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5497 - accuracy: 0.7315
Epoch 5/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5479 - accuracy: 0.7337
Epoch 6/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5459 - accuracy: 0.7345
Epoch 7/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5448 - accuracy: 0.7345
Epoch 8/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5432 - accuracy: 0.7375
Epoch 9/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5422 - accuracy: 0.7365
Epoch 10/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5422 - accuracy: 0.7376

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5451 - accuracy: 0.7332 - 496ms/epoch - 2ms/step
Loss: 0.5450962781906128, Accuracy: 0.7331778407096863


In [ ]:
# Export our model to HDF5 file
# nn3.save("AlphabetSoupCharity_Optimisation.h5")